In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline
life_expectancy = pd.read_table('./life_expectancy.csv', sep=',')

life_expectancy

### Exploratory Data Analysis

There are 2938 data entries, with data in 22 columns including life expectancy for a given year in a country, demographic variables, income composition, mortality rates and immunizations.

In [ ]:
print(f"Index: {life_expectancy.index} \n")
print(f"{life_expectancy.dtypes} \n")
print(f"Shape: {life_expectancy.shape}")

In [ ]:
life_expectancy['Status'] = life_expectancy['Status'].map({'Developed': 1, 'Developing': 0}) # change to numeric data

country_map = {}
count = 0
for country in life_expectancy['Country'].unique():
    count += 1
    
    country_map[country] = count
    
print(f"Country mappings: {country_map}")

life_expectancy['Country'] = life_expectancy['Country'].map(country_map) # change to numeric data

# remove empty spaces, upper case column names
life_expectancy.rename(columns={"under-five deaths": "under_five_deaths","Schooling": "schooling", "Income composition of resources": "income_composition", " BMI ": "bmi", " HIV/AIDS": "hiv_aids", " thinness  1-19 years" : "thinness_1_19", " thinness 5-9 years": "thinness_5_9", "Country": "country", "Year": "year", "Status": "developed", "Life expectancy ": "life_expectancy", "Adult Mortality": "adult_mortality", "infant deaths": "infant_deaths", "Alcohol": "alcohol", "percentage expenditure": "percentage_exp", "Hepatitis B": "hep_b", "Measles ": "measles", "Polio": "polio", "Total expenditure": "total_exp", "Diphtheria ": "diphtheria", "GDP": "gdp", "Population": "population"}, inplace=True)

The data looks at 193 countries, 161 of which are developing and 32 developed. All countries have data from 2000 to 2015. 10 rows of data are missing from 10 different developing countries in 2013.

Dropped all 10 missing entries for life expectancy as this is what we want to see whether the schooling feature has an impact.

In [ ]:
print(f" Number of countries: {len(life_expectancy['country'].unique())}")

life_expectancy[life_expectancy['developed'] == 0].groupby('country').mean()
life_expectancy[life_expectancy['developed'] == 1].groupby('country').mean()

life_expectancy.duplicated().unique() # no duplicate entries found

life_expectancy.groupby('year').mean() # all countries have data from 2000 - 2015. Some missing 2013

life_expectancy[life_expectancy['life_expectancy'].isna()] # 10 rows from life_expectancy are missing

In [ ]:
life_expectancy.dropna(subset=['life_expectancy'], inplace=True)
life_expectancy.shape

Schooling is the primary feature we want to see impacting the life expectancy of people in some way.

Schooling has 163 missing values. 3 removed by life expectancy clean-up. The other 160 missing values, were missing for all years (2000 - 2015) across 10 different countries. As these countries are missing schooling data from all years, these 10 countries will be dropped as no value can be obtained for the primary feature at this point.

In [ ]:
life_expectancy['schooling'].isna().sum() # 160 missing values for schooling, additional 3 removed by life_expectancy clean up

life_expectancy[life_expectancy['schooling'].isna()]
life_expectancy[life_expectancy['schooling'].isna()].groupby('country').mean()

life_expectancy.dropna(subset=['schooling'], inplace=True)

#### General Insights

2768 data entries

On average life expectancy is 69.3, min 36.3 and max 89
On average schooling 11.9 years, min 0 years and max 20.7 years

On average life expectancy in a developed country is 79.2 and in a developing county is 67.4
On average schooling in a developed country is 15.8 and in a developing county is 11.2

On avg. country 85 (Japan) has highest le, country 1 (Afghanistan) has lowest le
On avg. country 160 (South Sudan) has lowest schooling and country 8 (Australia) has highest

On avg. 6 years schooling 47.9 and 20.7 years, 86
On avg. at 55.2 year le you have 3.8 schooling and 82.8 you have 19.05

Lowest life expectancy 36.3 - 8.6 years of schooling (however there is only one instance of this)
Highest life expectancy 89.0 - 15+ to 20.3 years

On avg. gone from 66.9 le in 2000 to 71.7 in 2015

In [ ]:
life_expectancy.shape
life_expectancy.describe()

life_expectancy.groupby('developed')['life_expectancy'].mean()
life_expectancy.groupby('developed')['schooling'].mean()

life_expectancy.groupby('country')['life_expectancy'].mean().sort_values() # on avg. country 85 has highest le, country 1 has lowest le
life_expectancy.groupby('country')['schooling'].mean().sort_values() # on avg. country 160 has lowest schooling and country 8 has highest

life_expectancy.groupby('schooling')['life_expectancy'].mean().sort_values() # avg. 6 years schooling 47.9 and 20.7 years, 86
life_expectancy.groupby('life_expectancy')['schooling'].mean().sort_values() # avg. at 55.2 year le you have 3.8 schooling and 82.8 you have 19.05

life_expectancy['life_expectancy'].sort_values()
life_expectancy.loc[life_expectancy['life_expectancy'] == 36.3] # lowest life expectancy - 8.6 years of schooling
life_expectancy.loc[life_expectancy['life_expectancy'] == 89.0] # highest life expectancy - 15+ to 20.3 years

# is it better now in 2015 than in 2000?
life_expectancy.groupby('year')['life_expectancy'].mean() # avg. gone from 66.9 in 2000 to 71.7 in 2015

There is a a positive correlation between schooling and life expectancy. However it is clear that schooling is not the only thing that impacts life expectancy as with 0 hours schooling there is a range of life expectancy from late 40s to mid 70s. It's also clear that there is a wide range of life expectancy's below around 14 years of schooling, but this does reduce the longer someone is in schooling for. The range is approx. 20 years in life expectancy.

In [ ]:
life_expectancy.corr() # 75% correlation between life expectancy and schooling

In [ ]:
import seaborn as sns

schooling_df = life_expectancy[['schooling', 'life_expectancy']].copy()
sns.pairplot(schooling_df)

In [ ]:
sns.lmplot(x='schooling', y='life_expectancy', data=schooling_df, aspect=1.5, scatter_kws={'alpha':0.2});

#### General Clean-Up

under five deaths and infant deaths - under five deaths includes infant deaths;
thinness 1-19 and 5-9 - 1-19 include 5-9

In [ ]:
life_expectancy.drop(['country', 'year', 'developed'], axis='columns', inplace=True)
life_expectancy.drop(['thinness_5_9', 'infant_deaths'], axis='columns', inplace=True)

#### Finding control variables

Generally established that there is an association between schooling and association i.e. the more years attended at school the higher the life expectancy

Need to find control variables that have a correlation with both our primary feature and predictor to be included in the linear regression model for analysis in order to remove variable bias. 

In [ ]:
life_expectancy.corr() 

In [ ]:
plt.figure(figsize = (16,16))

sns.set_palette("coolwarm", 7)
sns.heatmap(life_expectancy.corr(), vmin=-1, vmax=1) 

Choosing adult mortality and income composition as the control features as they are correlated with both life expectancy and schooling (-0.6 and -0.4, 0.7 and 0.8, respectively).

#### General Information

New three features: Adult Mortality, Income Composition and Schooling;
Predictor: Life expectancy

In [ ]:
# adult Mortality Rates of both sexes (probability of dying between 15 and 60 years per 1000 population)
life_expectancy['adult_mortality'].describe()

In [ ]:
# human Development Index in terms of income composition of resources (index ranging from 0 to 1)
# http://hdr.undp.org/en/content/human-development-index-hdi
life_expectancy['income_composition'].describe()

In [ ]:
# number of years of Schooling(years)
life_expectancy['schooling'].describe()

In [ ]:
# add new control features
schooling_df['adult_mortality'] = life_expectancy['adult_mortality']
schooling_df['income_composition'] = life_expectancy['income_composition']

In [ ]:
fig, axs = plt.subplots(1, 3, sharey=True)

schooling_df.plot(kind='scatter', x='adult_mortality', y='life_expectancy', ax=axs[0], figsize=(10, 6));
schooling_df.plot(kind='scatter', x='income_composition', y='life_expectancy', ax=axs[1]);
schooling_df.plot(kind='scatter', x='schooling', y='life_expectancy', ax=axs[2]);

In [ ]:
sns.lmplot(x='adult_mortality', y='life_expectancy', data=schooling_df, aspect=1.5, scatter_kws={'alpha':0.2});

In [ ]:
sns.lmplot(x='income_composition', y='life_expectancy', data=schooling_df, aspect=1.5, scatter_kws={'alpha':0.2});

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True)

schooling_df.plot(kind='scatter', x='adult_mortality', y='schooling', ax=axs[0], figsize=(10, 6));
schooling_df.plot(kind='scatter', x='income_composition', y='schooling', ax=axs[1]);

In [ ]:
schooling_df.corr()

#### New Feature Clean-Up

In [ ]:
schooling_df['adult_mortality'].isna().unique()

In [ ]:
schooling_df['income_composition'].isna().unique()

There are cofounders that aren't taken into consideration in this dataset such as homelessness which could effect both life expectancy and years of schooling.

Features - Schooling, Adult Mortality and Income Composition
Predictor - Life Expectancy

Correlation of 75% between schooling and life_expectancy (Strong association) 
Consistent across different countries and different size populations
Specifically looking at the years of schooling taken
Logically as life expectancy increases it doesn't make sense that, that would increase someones years in schooling

Missing data not at random - from 10 countries 
No class imbalance as there is a wide range of life expectancies

###  Linear Regression Modelling

Testing and training on the same data set.

Regression Model fits the data to 73% - For the variation in life expectancy the features account for 73%
With 0 income_comp, adult_mortality and schooling - life expectancy is 54.5 years old

For every additional year of schooling the life expectancy of a person goes up 1.11 years

RMSE is 4.8 years off considerably more accurate than the baseline at 9.3 years.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np

feature_cols = ['income_composition', 'adult_mortality', 'schooling']
X = schooling_df[feature_cols]
y = schooling_df['life_expectancy']

lr = LinearRegression()
lr.fit(X, y)

print("Score: " + str(lr.score(X, y)))
print("Intercept: " + str(lr.intercept_))
print("Coeff: " + str(list(zip(feature_cols, lr.coef_))) + "\n")

y_pred_100_100 = lr.predict(X)
print('MSE: ' + str(metrics.mean_squared_error(y, y_pred_100_100))) 
print('MAE: ' + str(metrics.mean_absolute_error(y, y_pred_100_100))) 
print('RMSE: ' + str(np.sqrt(metrics.mean_squared_error(y, y_pred_100_100)))) 

Create a baseline to compare too

In [ ]:
schooling_df['life_expectancy'].mean()
schooling_df['life_expectancy_pred'] = y_pred_100_100
schooling_df['life_expectancy_base'] = schooling_df['life_expectancy'].mean()
print('MSE: ' + str(metrics.mean_squared_error(y, schooling_df['life_expectancy_base']))) # model considerably better than baseline
print('MAE: ' + str(metrics.mean_absolute_error(y, schooling_df['life_expectancy_base']))) 
print('RMSE: ' + str(np.sqrt(metrics.mean_squared_error(y, schooling_df['life_expectancy_base'])))) 

In [ ]:
sns.lmplot(x="schooling", y="life_expectancy", data=schooling_df)

In [ ]:
sns.lmplot(x="schooling", y="life_expectancy_pred", data=schooling_df)

In [ ]:
sns.lmplot(x="schooling", y="life_expectancy_base", data=schooling_df)

#### Consider 70/30, 80/20 and 90/10 Test-Train splits

In [ ]:
from sklearn.model_selection import train_test_split

def run_prediction(X_test, y_test, X_train, y_train):
    y_pred_test = lr.predict(X_test)
    y_pred_train = lr.predict(X_train)
    
    y_null = np.zeros_like(y_test, dtype=float)
    y_null.fill(y_test.mean())
    
    print("Train RMSE: " + str(np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))))
    print("Test RMSE: " + str(np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))))
    print("Base RMSE: " + str(np.sqrt(metrics.mean_squared_error(y_test, y_null))))
        
    plt.scatter(X_test['schooling'], y_pred_test) # dark blue
    plt.xlabel("Actual Schooling Years")
    plt.ylabel("Predicted Life expectancy")
    plt.title("Actual Schooling Years vs Predicted Life expectancy")
    
    plt.scatter(X_train['schooling'], y_pred_train) # light blue
    

def run_test_train_split(test_size, train_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, train_size=train_size, random_state=1)
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    print("Train Score: " + str(lr.score(X_train, y_train)))
    print("Test Score: " + str(lr.score(X_test, y_test)))
    print("Intercept: " + str(lr.intercept_))
    print("Coeff: " + str(list(zip(feature_cols, lr.coef_))) + '\n')
    
    run_prediction(X_test, y_test, X_train, y_train)

Test-Train 70:30
Fits slightly less on the test data 73.1% to 72.9% (Very close)
Coeff: 0.9 year per additional schooling year
Err slightly lower on test data 4.88 to 4.82
Still better than baseline at 9.35 err

Test-Tain 80:20
Fits better on the test data 72% to 73%
Coeff: 0.9 year per additional schooling year
Err reduced on test data 4.9 to 4.8 
Still better than baseline at 9.36 err

Test-Tain 90:10
Fits better on the train data 75% to 72% - slightly overfitted
Coeff: 0.6 year per additional schooling year
Err reduced on test data 4.68 to 4.85
Still better than baseline at 9.37 err

Lowest err on the test set is on the 80:20 split and the best fitted to the test data.

In [ ]:
run_test_train_split(0.7, 0.3)

In [ ]:
run_test_train_split(0.8, 0.2)

In [ ]:
run_test_train_split(0.9, 0.1)

#### K-Fold Validation

The best number of folds here is for 8-folds as it has the lowest error of 4.81 on average and has an average fit of 73%. Using Test-Tain split at 80:20 has a very similar result to the k-fold average.

In [ ]:
from sklearn import model_selection

kf = model_selection.KFold(n_splits=8, shuffle=True, random_state=1) # manually update values 5 to 10
rmse_values = []
scores = []
n = 0

print("~~~~ CROSS VALIDATION each fold ~~~~")
for train_index, test_index in kf.split(X, y):
    lr = LinearRegression().fit(X.iloc[train_index], y.iloc[train_index])
    
    rmse_values.append(np.sqrt(metrics.mean_squared_error(y.iloc[test_index], lr.predict(X.iloc[test_index]))))
    scores.append(lr.score(X, y))
    
    n += 1
    
    print('Model {}'.format(n))
    print('MSE: {}'.format(rmse_values[n-1]))
    print('R2: {}\n'.format(scores[n-1]))


print("~~~~ SUMMARY OF CROSS VALIDATION ~~~~")
print('Mean of RMSE for all folds: {}'.format(np.mean(rmse_values)))
print('Mean of R2 for all folds: {}'.format(np.mean(scores)))

#### Removing Outliers

Looking at the Scatter Graph the outliers here appear to be the points at schooling at 0 years and life_expectancy at 35 with shooling of 8 years.

In [ ]:
sns.lmplot(x="schooling", y="life_expectancy", data=schooling_df)

In [ ]:
schooling_df.drop(['life_expectancy_pred', 'life_expectancy_base'], axis='columns', inplace=True)

In [ ]:
from scipy import stats
schooling_df[(np.abs(stats.zscore(schooling_df)) < 4).all(axis=1)] # remove anything below the threshhold of the z-score for each col

In [ ]:
run_test_train_split(0.8, 0.2) # no significant impact